In [ ]:
import pygame
import numpy as np
from scipy.linalg import cholesky

# (위에 정의된 상세 주석 버전의 UnscentedKalmanFilter 클래스가 여기에 위치합니다)
class UnscentedKalmanFilter:
    """
    1D 시스템을 위한 무향 칼만 필터 구현
    상태 벡터: [위치, 속도]
    """
    def __init__(self, x0, P, Q, R, dt, fx, hx):
        # --- 1. 초기화 ---
        self.x = np.array(x0)  # 상태 변수 초기값 x_0
        self.P = np.array(P)   # 상태 공분산 행렬 초기값 P_0
        self.Q = np.array(Q)   # 프로세스 노이즈 공분산 Q
        self.R = np.array(R)   # 측정 노이즈 공분산 R
        self.dt = dt           # 시간 간격 delta_t
        self.fx = fx           # 비선형 상태 전이 함수 f(x)
        self.hx = hx           # 측정 함수 h(x)

        self.n = len(x0)       # 상태 변수의 차원 (n)
        self.m = self.R.shape[0] # 측정 변수의 차원

        # --- 2. UKF 파라미터 및 가중치 계산 ---
        # 시그마 포인트의 분포를 결정하는 파라미터 (alpha, beta, kappa)
        alpha, beta, kappa = 1e-3, 2, 0
        
        # 스케일링 파라미터 람다(λ) 계산
        # λ = α²(n + κ) - n
        self.lambda_ = alpha**2 * (self.n + kappa) - self.n
        
        # 시그마 포인트들의 가중치(weight) 계산
        # 평균 계산을 위한 가중치 Wm (Weights for mean)
        # 공분산 계산을 위한 가중치 Wc (Weights for covariance)
        self.Wm = np.full(2 * self.n + 1, 0.5 / (self.n + self.lambda_))
        self.Wc = np.full(2 * self.n + 1, 0.5 / (self.n + self.lambda_))
        
        # 첫 번째 시그마 포인트(평균점)의 가중치는 따로 계산
        # Wm[0] = λ / (n + λ)
        # Wc[0] = λ / (n + λ) + (1 - α² + β)
        self.Wm[0] = self.lambda_ / (self.n + self.lambda_)
        self.Wc[0] = self.Wm[0] + (1 - alpha**2 + beta)

    def _generate_sigma_points(self):
        """ 
        현재 상태(x)와 공분산(P)을 대표하는 2n+1개의 시그마 포인트를 생성합니다.
        수식: 
            X_0 = x
            X_i = x + [√( (n+λ)P )]_i  for i=1,...,n
            X_i = x - [√( (n+λ)P )]_{i-n} for i=n+1,...,2n
        여기서 √(...)는 촐레스키 분해를 통해 계산합니다.
        """
        sigma_points = np.zeros((2 * self.n + 1, self.n))
        
        # 공분산 행렬의 제곱근 계산 (촐레스키 분해)
        try:
            U = cholesky((self.n + self.lambda_) * self.P)
        except np.linalg.LinAlgError:
            # 행렬이 Positive Definite가 아닐 경우 대비
            U = np.zeros_like(self.P)

        sigma_points[0] = self.x # 첫 번째 포인트는 현재 상태의 평균
        for i in range(self.n):
            sigma_points[i + 1]       = self.x + U[i]
            sigma_points[i + self.n + 1] = self.x - U[i]
            
        return sigma_points

    def predict(self, u):
        """ ## 예측 (Predict / Time Update) ## """
        
        # --- 1. 시그마 포인트 생성 (X_k-1|k-1) ---
        sigma_points = self._generate_sigma_points()
        
        # --- 2. 시그마 포인트 전파 (Propagate) ---
        # 각 시그마 포인트를 비선형 상태 전이 함수 f(x)에 통과시킵니다.
        # X*_k|k-1 = f(X_k-1|k-1, u_k)
        self.sigma_points_pred = np.array([self.fx(sp, u, self.dt) for sp in sigma_points])
        
        # --- 3. 예측된 상태(Prior) 계산 ---
        # 전파된 시그마 포인트들의 가중 평균을 계산합니다.
        # x̂_k|k-1 = Σ(i=0 to 2n) [Wm_i * X*_k|k-1_i]
        self.x_pred = np.dot(self.Wm, self.sigma_points_pred)
        
        # --- 4. 예측된 공분산(Prior) 계산 ---
        # 전파된 시그마 포인트들의 가중 공분산을 계산하고 프로세스 노이즈(Q)를 더합니다.
        # P_k|k-1 = Σ(i=0 to 2n) [Wc_i * (X*_k|k-1_i - x̂_k|k-1)(...)^T] + Q
        P_pred = np.zeros((self.n, self.n))
        for i in range(2 * self.n + 1):
            y = self.sigma_points_pred[i] - self.x_pred
            P_pred += self.Wc[i] * np.outer(y, y)
        self.P_pred = P_pred + self.Q
        
        # 예측된 값을 현재 필터의 상태로 저장
        self.x = self.x_pred
        self.P = self.P_pred

    def update(self, z):
        """ ## 업데이트 (Update / Measurement Update) ## """

        # --- 1. 예측된 시그마 포인트를 측정 공간으로 변환 ---
        # 예측 단계에서 계산한 시그마 포인트(X*_k|k-1)를 측정 함수 h(x)에 통과시킵니다.
        # Z_k|k-1 = h(X*_k|k-1)
        sigma_points_meas = np.array([self.hx(sp) for sp in self.sigma_points_pred])
        
        # --- 2. 예측된 측정값 계산 ---
        # 변환된 측정 시그마 포인트들의 가중 평균을 계산합니다.
        # ẑ_k|k-1 = Σ(i=0 to 2n) [Wm_i * Z_k|k-1_i]
        z_pred = np.dot(self.Wm, sigma_points_meas)
        
        # --- 3. 예측 오차 공분산(S_k) 및 교차 공분산(P_xzk) 계산 ---
        # 예측 오차(Innovation) 공분산 S_k = P_zz
        # S_k = Σ(i=0 to 2n) [Wc_i * (Z_k|k-1_i - ẑ_k|k-1)(...)^T] + R
        P_zz = np.zeros((self.m, self.m))
        # 상태-측정 교차 공분산 P_xzk
        # P_xzk = Σ(i=0 to 2n) [Wc_i * (X*_k|k-1_i - x̂_k|k-1)(Z_k|k-1_i - ẑ_k|k-1)^T]
        P_xz = np.zeros((self.n, self.m))
        
        for i in range(2 * self.n + 1):
            res_z = (sigma_points_meas[i] - z_pred).reshape(self.m, 1)
            res_x = (self.sigma_points_pred[i] - self.x_pred).reshape(self.n, 1)
            P_zz += self.Wc[i] * res_z @ res_z.T
            P_xz += self.Wc[i] * res_x @ res_z.T
        P_zz += self.R
        
        # --- 4. 칼만 이득(K_k) 계산 ---
        # K_k = P_xzk * S_k⁻¹
        K = P_xz @ np.linalg.inv(P_zz)
        
        # --- 5. 최종 상태(Posterior) 업데이트 ---
        # 실제 측정값(z)과 예측된 측정값의 차이(innovation)를 이용합니다.
        # x̂_k|k = x̂_k|k-1 + K_k * (z_k - ẑ_k|k-1)
        innovation = (z - z_pred).reshape(self.m, 1)
        self.x = self.x_pred + (K @ innovation).flatten()
        
        # --- 6. 최종 공분산(Posterior) 업데이트 ---
        # P_k|k = P_k|k-1 - K_k * S_k * K_k^T
        self.P = self.P_pred - K @ P_zz @ K.T

# --- 시스템 모델 (이전과 동일) ---
def state_transition_function(x, u, dt, k_d):
    pos, vel = x
    accel_cmd = u
    
    accel_drag = -k_d * vel * abs(vel)
    accel_net = accel_cmd + accel_drag
    
    pos_next = pos + vel * dt + 0.5 * accel_net * dt**2
    vel_next = vel + accel_net * dt
    
    return np.array([pos_next, vel_next])

def measurement_function(x):
    return np.array([x[0]])

# --- Pygame 및 시뮬레이션 파라미터 설정 (요청대로 수정) ---
pygame.init()
WIDTH, HEIGHT = 1000, 400
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("1D UKF 시뮬레이션 (일정 제어 입력 + 공기 저항)")
clock = pygame.time.Clock()

# 수정된 상수 값
dt = 0.1
accel_noise_std = 0.2
measurement_noise_std = 8.0
num_steps = 500
K_DRAG = 0.003
CMD_A = 10.0 

# 시스템 노이즈 행렬
Q_diag = np.array([
    (0.5 * dt**2)**2 * accel_noise_std**2,
    (dt)**2 * accel_noise_std**2
])
Q = np.diag(Q_diag)
R = np.array([[measurement_noise_std**2]])

# 초기 상태 및 추정값
initial_true_state = np.array([50.0, 0.0])
x_est_initial = np.array([30.0, 5.0])
P_initial = np.eye(2) * 50.0

# ==============================================================
# 단계 1: 실제 경로 시뮬레이션
# ==============================================================
print("단계 1: 실제 경로 시뮬레이션 중...")
true_states = []
current_true_state = initial_true_state.copy()
control_input = CMD_A

for _ in range(num_steps):
    process_noise = np.random.multivariate_normal(np.zeros(2), Q)
    current_true_state = state_transition_function(current_true_state, control_input, dt, K_DRAG) + process_noise
    true_states.append(current_true_state.copy())

# ==============================================================
# 단계 2: UKF 적용
# ==============================================================
print("단계 2: UKF 적용 중...")
ukf = UnscentedKalmanFilter(
    x0=x_est_initial, P=P_initial, Q=Q, R=R, dt=dt,
    fx=lambda x, u, d: state_transition_function(x, u, d, K_DRAG),
    hx=measurement_function
)

estimated_states = []
measurements_for_viz = []
control_input = CMD_A 

for k in range(num_steps):
    true_state_k = true_states[k]
    
    ukf.predict(u=control_input)
    
    measurement_noise = np.random.normal(0, measurement_noise_std)
    measurement_k = measurement_function(true_state_k) + measurement_noise
    measurements_for_viz.append(measurement_k.copy())
    
    ukf.update(measurement_k)
    estimated_states.append(ukf.x.copy())

# ==============================================================
# 단계 3: 시각화
# ==============================================================
print("단계 3: 시각화 시작...")
running = True
frame_index = 0
y_pos = HEIGHT // 2

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill((255, 255, 255))

    if frame_index < num_steps:
        true_state_k = true_states[frame_index]
        measurement_k = measurements_for_viz[frame_index]
        estimated_state_k = estimated_states[frame_index]

        if frame_index > 0:
            prev_true_x = int(np.clip(true_states[frame_index-1][0], 0, WIDTH))
            prev_est_x = int(np.clip(estimated_states[frame_index-1][0], 0, WIDTH))
            pygame.draw.line(screen, (0, 0, 200), (prev_true_x, y_pos), (int(true_state_k[0]), y_pos), 2)
            pygame.draw.line(screen, (200, 0, 0), (prev_est_x, y_pos), (int(estimated_state_k[0]), y_pos), 2)

        true_x = int(np.clip(true_state_k[0], 0, WIDTH))
        pygame.draw.circle(screen, (0, 0, 255), (true_x, y_pos), 8)

        meas_x = int(np.clip(measurement_k[0], 0, WIDTH))
        # pygame.draw.circle(screen, (180, 180, 180), (meas_x, y_pos), 6)

        est_x = int(np.clip(estimated_state_k[0], 0, WIDTH))
        pygame.draw.circle(screen, (255, 0, 0), (est_x, y_pos), 8, 3)

        font = pygame.font.Font(None, 30)
        text_step = font.render(f"스텝: {frame_index+1}/{num_steps} (UKF, 일정 CMD)", True, (0, 0, 0))
        screen.blit(text_step, (10, 10))
        
        text_drag = font.render(f"K_DRAG: {K_DRAG:.3f}", True, (0,0,0))
        screen.blit(text_drag, (10, 40))
        
        text_cmd = font.render(f"CMD_A: {abs(CMD_A):.2f}", True, (0,0,0))
        screen.blit(text_cmd, (10, 70))
        
        frame_index += 1
    else:
        font = pygame.font.Font(None, 50)
        text_done = font.render("UKF 시뮬레이션 완료", True, (0, 100, 0))
        text_rect = text_done.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(text_done, text_rect)

    pygame.display.flip()
    clock.tick(30)

pygame.quit()
print("시각화 종료 및 Pygame 종료됨.")

단계 1: 실제 경로 시뮬레이션 중...
단계 2: UKF 적용 중...
단계 3: 시각화 시작...
시각화 종료 및 Pygame 종료됨.


In [16]:
import pygame
import numpy as np
from scipy.linalg import cholesky

# UnscentedKalmanFilter 클래스는 변경 없이 그대로 사용됩니다.
class UnscentedKalmanFilter:
    def __init__(self, x0, P, Q, R, dt, fx, hx):
        self.x = np.array(x0)
        self.P = np.array(P)
        self.Q = np.array(Q)
        self.R = np.array(R)
        self.dt = dt
        self.fx = fx
        self.hx = hx
        self.n = len(x0)
        self.m = self.R.shape[0]
        alpha, beta, kappa = 1e-3, 2, 0
        self.lambda_ = alpha**2 * (self.n + kappa) - self.n
        self.Wm = np.full(2 * self.n + 1, 0.5 / (self.n + self.lambda_))
        self.Wc = np.full(2 * self.n + 1, 0.5 / (self.n + self.lambda_))
        self.Wm[0] = self.lambda_ / (self.n + self.lambda_)
        self.Wc[0] = self.Wm[0] + (1 - alpha**2 + beta)

    def _generate_sigma_points(self):
        sigma_points = np.zeros((2 * self.n + 1, self.n))
        try:
            U = cholesky((self.n + self.lambda_) * self.P)
        except np.linalg.LinAlgError:
            U = np.zeros_like(self.P)
        sigma_points[0] = self.x
        for i in range(self.n):
            sigma_points[i + 1] = self.x + U[i]
            sigma_points[i + self.n + 1] = self.x - U[i]
        return sigma_points

    def predict(self, u):
        sigma_points = self._generate_sigma_points()
        self.sigma_points_pred = np.array([self.fx(sp, u, self.dt) for sp in sigma_points])
        self.x_pred = np.dot(self.Wm, self.sigma_points_pred)
        P_pred = np.zeros((self.n, self.n))
        for i in range(2 * self.n + 1):
            y = self.sigma_points_pred[i] - self.x_pred
            P_pred += self.Wc[i] * np.outer(y, y)
        self.P_pred = P_pred + self.Q
        self.x = self.x_pred
        self.P = self.P_pred

    def update(self, z):
        sigma_points_meas = np.array([self.hx(sp) for sp in self.sigma_points_pred])
        z_pred = np.dot(self.Wm, sigma_points_meas)
        P_zz = np.zeros((self.m, self.m))
        P_xz = np.zeros((self.n, self.m))
        for i in range(2 * self.n + 1):
            res_z = (sigma_points_meas[i] - z_pred).reshape(self.m, 1)
            res_x = (self.sigma_points_pred[i] - self.x_pred).reshape(self.n, 1)
            P_zz += self.Wc[i] * res_z @ res_z.T
            P_xz += self.Wc[i] * res_x @ res_z.T
        P_zz += self.R
        K = P_xz @ np.linalg.inv(P_zz)
        innovation = (z - z_pred).reshape(self.m, 1)
        self.x = self.x_pred + (K @ innovation).flatten()
        self.P = self.P_pred - K @ P_zz @ K.T

# 2D 시스템 모델 (변경 없음)
def state_transition_function(x, u, dt, k_d):
    px, py, vx, vy = x
    ax_cmd, ay_cmd = u
    speed = np.sqrt(vx**2 + vy**2)
    if speed < 1e-6:
        ax_drag, ay_drag = 0, 0
    else:
        ax_drag = -k_d * vx * speed
        ay_drag = -k_d * vy * speed
    ax_net = ax_cmd + ax_drag
    ay_net = ay_cmd + ay_drag
    px_next = px + vx * dt + 0.5 * ax_net * dt**2
    py_next = py + vy * dt + 0.5 * ay_net * dt**2
    vx_next = vx + ax_net * dt
    vy_next = vy + ay_net * dt
    return np.array([px_next, py_next, vx_next, vy_next])

def measurement_function(x):
    return np.array([x[0], x[1]])

# --- Pygame 및 2D 시뮬레이션 파라미터 설정 ---
pygame.init()
WIDTH, HEIGHT = 1000, 800
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("2D UKF 시뮬레이션 (지그재그 경로)")
clock = pygame.time.Clock()

dt = 0.1
accel_noise_std = 0.5
measurement_noise_std = 10.0
num_steps = 1000
K_DRAG = 0.005
ZIGZAG_INTERVAL = 70  # 지그재그 방향 전환 간격 (스텝 단위)

# 2D 제어 입력 (명령 가속도)
CMD_A = np.array([5.0, 8.0])

# 노이즈 행렬 (변경 없음)
Q_diag = np.array([(0.5*dt**2)**2*accel_noise_std**2, (0.5*dt**2)**2*accel_noise_std**2, (dt)**2*accel_noise_std**2, (dt)**2*accel_noise_std**2])
Q = np.diag(Q_diag)
R = np.eye(2) * measurement_noise_std**2

# 초기 상태 (변경 없음)
initial_true_state = np.array([WIDTH / 8, HEIGHT / 2, 5.0, 0.0])
x_est_initial = np.array([initial_true_state[0] + 30, initial_true_state[1] - 30, 0.0, 0.0])
P_initial = np.eye(4) * 500.0

# ==============================================================
# 단계 1: 실제 경로 시뮬레이션 (지그재그 로직 추가)
# ==============================================================
print("단계 1: 실제 경로 시뮬레이션 중...")
true_states = []
current_true_state = initial_true_state.copy()
control_input = CMD_A.copy() # 제어 입력을 복사하여 사용

for k in range(num_steps):
    # ZIGZAG_INTERVAL 마다 제어 입력의 y 방향을 뒤집음
    if k > 0 and k % ZIGZAG_INTERVAL == 0:
        control_input[1] *= -1

    process_noise = np.random.multivariate_normal(np.zeros(4), Q)
    current_true_state = state_transition_function(current_true_state, control_input, dt, K_DRAG) + process_noise
    true_states.append(current_true_state.copy())

# ==============================================================
# 단계 2: UKF 적용 (지그재그 로직 추가)
# ==============================================================
print("단계 2: UKF 적용 중...")
ukf = UnscentedKalmanFilter(
    x0=x_est_initial, P=P_initial, Q=Q, R=R, dt=dt,
    fx=lambda x, u, d: state_transition_function(x, u, d, K_DRAG),
    hx=measurement_function
)
estimated_states = []
measurements_for_viz = []
control_input = CMD_A.copy() # 제어 입력을 다시 초기화

for k in range(num_steps):
    # 실제 경로와 동일한 제어 입력 로직을 필터에도 적용
    if k > 0 and k % ZIGZAG_INTERVAL == 0:
        control_input[1] *= -1

    true_state_k = true_states[k]
    ukf.predict(u=control_input) # 변경된 제어 입력을 전달
    measurement_noise = np.random.multivariate_normal(np.zeros(2), R)
    measurement_k = measurement_function(true_state_k) + measurement_noise
    measurements_for_viz.append(measurement_k.copy())
    ukf.update(measurement_k)
    estimated_states.append(ukf.x.copy())

# ==============================================================
# 단계 3: 2D 시각화 (변경 없음)
# ==============================================================
print("단계 3: 시각화 시작...")
running = True
frame_index = 0

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill((255, 255, 255))

    if frame_index < num_steps:
        true_state_k = true_states[frame_index]
        measurement_k = measurements_for_viz[frame_index]
        estimated_state_k = estimated_states[frame_index]

        if frame_index > 0:
            prev_true_pos = (int(true_states[frame_index-1][0]), int(true_states[frame_index-1][1]))
            curr_true_pos = (int(true_state_k[0]), int(true_state_k[1]))
            pygame.draw.line(screen, (0, 0, 200), prev_true_pos, curr_true_pos, 2)
            prev_est_pos = (int(estimated_states[frame_index-1][0]), int(estimated_states[frame_index-1][1]))
            curr_est_pos = (int(estimated_state_k[0]), int(estimated_state_k[1]))
            pygame.draw.line(screen, (200, 0, 0), prev_est_pos, curr_est_pos, 2)

        true_pos = (int(true_state_k[0]), int(true_state_k[1]))
        meas_pos = (int(measurement_k[0]), int(measurement_k[1]))
        est_pos = (int(estimated_state_k[0]), int(estimated_state_k[1]))

        pygame.draw.circle(screen, (180, 180, 180), meas_pos, 6)
        pygame.draw.circle(screen, (0, 0, 255), true_pos, 8)
        pygame.draw.circle(screen, (255, 0, 0), est_pos, 8, 3)

        font = pygame.font.Font(None, 30)
        text_step = font.render(f"스텝: {frame_index+1}/{num_steps}", True, (0,0,0))
        screen.blit(text_step, (10, 10))
        # 현재 적용되는 제어 입력을 표시하기 위해 로직 수정
        current_cmd = CMD_A.copy()
        num_flips = frame_index // ZIGZAG_INTERVAL
        current_cmd[1] *= (-1)**num_flips
        text_cmd = font.render(f"CMD_A: [{current_cmd[0]:.1f}, {current_cmd[1]:.1f}]", True, (0,0,0))
        screen.blit(text_cmd, (10, 40))

        frame_index += 1
    else:
        font = pygame.font.Font(None, 50)
        text_done = font.render("2D UKF 시뮬레이션 완료", True, (0, 100, 0))
        text_rect = text_done.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(text_done, text_rect)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()
print("시각화 종료 및 Pygame 종료됨.")

단계 1: 실제 경로 시뮬레이션 중...
단계 2: UKF 적용 중...
단계 3: 시각화 시작...
시각화 종료 및 Pygame 종료됨.


In [15]:
import pygame
import numpy as np
from scipy.linalg import cholesky

# UnscentedKalmanFilter 클래스는 변경 없이 그대로 사용됩니다.
class UnscentedKalmanFilter:
    def __init__(self, x0, P, Q, R, dt, fx, hx):
        self.x = np.array(x0)
        self.P = np.array(P)
        self.Q = np.array(Q)
        self.R = np.array(R)
        self.dt = dt
        self.fx = fx
        self.hx = hx
        self.n = len(x0)
        self.m = self.R.shape[0]
        alpha, beta, kappa = 1e-3, 2, 0
        self.lambda_ = alpha**2 * (self.n + kappa) - self.n
        self.Wm = np.full(2 * self.n + 1, 0.5 / (self.n + self.lambda_))
        self.Wc = np.full(2 * self.n + 1, 0.5 / (self.n + self.lambda_))
        self.Wm[0] = self.lambda_ / (self.n + self.lambda_)
        self.Wc[0] = self.Wm[0] + (1 - alpha**2 + beta)

    def _generate_sigma_points(self):
        sigma_points = np.zeros((2 * self.n + 1, self.n))
        try:
            U = cholesky((self.n + self.lambda_) * self.P)
        except np.linalg.LinAlgError:
            U = np.zeros_like(self.P)
        sigma_points[0] = self.x
        for i in range(self.n):
            sigma_points[i + 1] = self.x + U[i]
            sigma_points[i + self.n + 1] = self.x - U[i]
        return sigma_points

    def predict(self, u):
        sigma_points = self._generate_sigma_points()
        self.sigma_points_pred = np.array([self.fx(sp, u, self.dt) for sp in sigma_points])
        self.x_pred = np.dot(self.Wm, self.sigma_points_pred)
        P_pred = np.zeros((self.n, self.n))
        for i in range(2 * self.n + 1):
            y = self.sigma_points_pred[i] - self.x_pred
            P_pred += self.Wc[i] * np.outer(y, y)
        self.P_pred = P_pred + self.Q
        self.x = self.x_pred
        self.P = self.P_pred

    def update(self, z):
        sigma_points_meas = np.array([self.hx(sp) for sp in self.sigma_points_pred])
        z_pred = np.dot(self.Wm, sigma_points_meas)
        P_zz = np.zeros((self.m, self.m))
        P_xz = np.zeros((self.n, self.m))
        for i in range(2 * self.n + 1):
            res_z = (sigma_points_meas[i] - z_pred).reshape(self.m, 1)
            res_x = (self.sigma_points_pred[i] - self.x_pred).reshape(self.n, 1)
            P_zz += self.Wc[i] * res_z @ res_z.T
            P_xz += self.Wc[i] * res_x @ res_z.T
        P_zz += self.R
        K = P_xz @ np.linalg.inv(P_zz)
        innovation = (z - z_pred).reshape(self.m, 1)
        self.x = self.x_pred + (K @ innovation).flatten()
        self.P = self.P_pred - K @ P_zz @ K.T

# 2D 시스템 모델 (변경 없음)
def state_transition_function(x, u, dt, k_d):
    px, py, vx, vy = x
    ax_cmd, ay_cmd = u
    speed = np.sqrt(vx**2 + vy**2)
    if speed < 1e-6:
        ax_drag, ay_drag = 0, 0
    else:
        ax_drag = -k_d * vx * speed
        ay_drag = -k_d * vy * speed
    ax_net = ax_cmd + ax_drag
    ay_net = ay_cmd + ay_drag
    px_next = px + vx * dt + 0.5 * ax_net * dt**2
    py_next = py + vy * dt + 0.5 * ay_net * dt**2
    vx_next = vx + ax_net * dt
    vy_next = vy + ay_net * dt
    return np.array([px_next, py_next, vx_next, vy_next])

def measurement_function(x):
    return np.array([x[0], x[1]])

# --- Pygame 및 2D 시뮬레이션 파라미터 설정 ---
pygame.init()
WIDTH, HEIGHT = 1000, 800
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("2D UKF 시뮬레이션 (속도 급변 경로)")
clock = pygame.time.Clock()

dt = 0.1
accel_noise_std = 2.0
measurement_noise_std = 2.0
num_steps = 1000
K_DRAG = 0.01
SHARP_TURN_INTERVAL = 100  # 속도 급변 간격 (스텝 단위)

# 제어 입력은 이제 일정하게 유지
CMD_A = np.array([0.5, 0.5]) 
# 턴할 때마다 번갈아 적용될 속도 벡터들
TURN_VELOCITY_VECTORS = [
    np.array([10.0, 40.0]),
    np.array([40.0, -10.0])
]

# 노이즈 행렬 (변경 없음)
Q_diag = np.array([(0.5*dt**2)**2*accel_noise_std**2, (0.5*dt**2)**2*accel_noise_std**2, (dt)**2*accel_noise_std**2, (dt)**2*accel_noise_std**2])
Q = np.diag(Q_diag)
R = np.eye(2) * measurement_noise_std**2

# 초기 상태
initial_true_state = np.array([WIDTH / 8, HEIGHT / 2, 40.0, 0.0])
x_est_initial = np.array([initial_true_state[0] + 30, initial_true_state[1] - 30, 0.0, 0.0])
P_initial = np.eye(4) * 500.0

# ==============================================================
# 단계 1: 실제 경로 시뮬레이션 (속도 직접 조작)
# ==============================================================
print("단계 1: 실제 경로 시뮬레이션 중...")
true_states = []
current_true_state = initial_true_state.copy()
turn_index = 0

for k in range(num_steps):
    # 물리 모델에 따라 다음 상태 계산
    process_noise = np.random.multivariate_normal(np.zeros(4), Q)
    current_true_state = state_transition_function(current_true_state, CMD_A, dt, K_DRAG) + process_noise

    # SHARP_TURN_INTERVAL 마다 속도를 강제로 변경
    if k > 0 and k % SHARP_TURN_INTERVAL == 0:
        new_velocity = TURN_VELOCITY_VECTORS[turn_index % len(TURN_VELOCITY_VECTORS)]
        current_true_state[2] = new_velocity[0] # vx 변경
        current_true_state[3] = new_velocity[1] # vy 변경
        turn_index += 1
        
    true_states.append(current_true_state.copy())

# ==============================================================
# 단계 2: UKF 적용 (필터는 속도 변화를 모름)
# ==============================================================
print("단계 2: UKF 적용 중...")
ukf = UnscentedKalmanFilter(
    x0=x_est_initial, P=P_initial, Q=Q, R=R, dt=dt,
    fx=lambda x, u, d: state_transition_function(x, u, d, K_DRAG),
    hx=measurement_function
)
estimated_states = []
measurements_for_viz = []

for k in range(num_steps):
    true_state_k = true_states[k]
    # 필터는 일정한 CMD_A가 계속 작용한다고 믿고 예측
    ukf.predict(u=CMD_A)
    
    # 실제 경로에서 생성된 측정값으로 업데이트 (여기서 큰 오차 발생)
    measurement_noise = np.random.multivariate_normal(np.zeros(2), R)
    measurement_k = measurement_function(true_state_k) + measurement_noise
    measurements_for_viz.append(measurement_k.copy())
    ukf.update(measurement_k)
    estimated_states.append(ukf.x.copy())

# ==============================================================
# 단계 3: 2D 시각화 (변경 없음)
# ==============================================================
print("단계 3: 시각화 시작...")
running = True
frame_index = 0

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill((255, 255, 255))

    if frame_index < num_steps:
        true_state_k = true_states[frame_index]
        measurement_k = measurements_for_viz[frame_index]
        estimated_state_k = estimated_states[frame_index]

        if frame_index > 0:
            prev_true_pos = (int(true_states[frame_index-1][0]), int(true_states[frame_index-1][1]))
            curr_true_pos = (int(true_state_k[0]), int(true_state_k[1]))
            pygame.draw.line(screen, (0, 0, 200), prev_true_pos, curr_true_pos, 2)
            prev_est_pos = (int(estimated_states[frame_index-1][0]), int(estimated_states[frame_index-1][1]))
            curr_est_pos = (int(estimated_state_k[0]), int(estimated_state_k[1]))
            pygame.draw.line(screen, (200, 0, 0), prev_est_pos, curr_est_pos, 2)

        true_pos = (int(true_state_k[0]), int(true_state_k[1]))
        meas_pos = (int(measurement_k[0]), int(measurement_k[1]))
        est_pos = (int(estimated_state_k[0]), int(estimated_state_k[1]))

        pygame.draw.circle(screen, (180, 180, 180), meas_pos, 6)
        pygame.draw.circle(screen, (0, 0, 255), true_pos, 8)
        pygame.draw.circle(screen, (255, 0, 0), est_pos, 8, 3)

        font = pygame.font.Font(None, 30)
        text_step = font.render(f"스텝: {frame_index+1}/{num_steps}", True, (0,0,0))
        screen.blit(text_step, (10, 10))
        text_cmd = font.render(f"CMD_A: [{CMD_A[0]:.1f}, {CMD_A[1]:.1f}]", True, (0,0,0))
        screen.blit(text_cmd, (10, 40))

        frame_index += 1
    else:
        font = pygame.font.Font(None, 50)
        text_done = font.render("2D UKF 시뮬레이션 완료", True, (0, 100, 0))
        text_rect = text_done.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(text_done, text_rect)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()
print("시각화 종료 및 Pygame 종료됨.")

단계 1: 실제 경로 시뮬레이션 중...
단계 2: UKF 적용 중...
단계 3: 시각화 시작...
시각화 종료 및 Pygame 종료됨.


In [12]:
import pygame
import numpy as np
from scipy.linalg import cholesky

# UnscentedKalmanFilter 클래스는 변경 없이 그대로 사용됩니다.
class UnscentedKalmanFilter:
    def __init__(self, x0, P, Q, R, dt, fx, hx):
        self.x = np.array(x0)
        self.P = np.array(P)
        self.Q = np.array(Q)
        self.R = np.array(R)
        self.dt = dt
        self.fx = fx
        self.hx = hx
        self.n = len(x0)
        self.m = self.R.shape[0]
        alpha, beta, kappa = 1e-3, 2, 0
        self.lambda_ = alpha**2 * (self.n + kappa) - self.n
        self.Wm = np.full(2 * self.n + 1, 0.5 / (self.n + self.lambda_))
        self.Wc = np.full(2 * self.n + 1, 0.5 / (self.n + self.lambda_))
        self.Wm[0] = self.lambda_ / (self.n + self.lambda_)
        self.Wc[0] = self.Wm[0] + (1 - alpha**2 + beta)

    def _generate_sigma_points(self):
        sigma_points = np.zeros((2 * self.n + 1, self.n))
        try:
            U = cholesky((self.n + self.lambda_) * self.P)
        except np.linalg.LinAlgError:
            U = np.zeros_like(self.P)
        sigma_points[0] = self.x
        for i in range(self.n):
            sigma_points[i + 1] = self.x + U[i]
            sigma_points[i + self.n + 1] = self.x - U[i]
        return sigma_points

    def predict(self, u):
        sigma_points = self._generate_sigma_points()
        self.sigma_points_pred = np.array([self.fx(sp, u, self.dt) for sp in sigma_points])
        self.x_pred = np.dot(self.Wm, self.sigma_points_pred)
        P_pred = np.zeros((self.n, self.n))
        for i in range(2 * self.n + 1):
            y = self.sigma_points_pred[i] - self.x_pred
            P_pred += self.Wc[i] * np.outer(y, y)
        self.P_pred = P_pred + self.Q
        self.x = self.x_pred
        self.P = self.P_pred

    def update(self, z):
        sigma_points_meas = np.array([self.hx(sp) for sp in self.sigma_points_pred])
        z_pred = np.dot(self.Wm, sigma_points_meas)
        P_zz = np.zeros((self.m, self.m))
        P_xz = np.zeros((self.n, self.m))
        for i in range(2 * self.n + 1):
            res_z = (sigma_points_meas[i] - z_pred).reshape(self.m, 1)
            res_x = (self.sigma_points_pred[i] - self.x_pred).reshape(self.n, 1)
            P_zz += self.Wc[i] * res_z @ res_z.T
            P_xz += self.Wc[i] * res_x @ res_z.T
        P_zz += self.R
        K = P_xz @ np.linalg.inv(P_zz)
        innovation = (z - z_pred).reshape(self.m, 1)
        self.x = self.x_pred + (K @ innovation).flatten()
        self.P = self.P_pred - K @ P_zz @ K.T

# 2D 시스템 모델 (변경 없음)
def state_transition_function(x, u, dt, k_d):
    px, py, vx, vy = x; ax_cmd, ay_cmd = u
    speed = np.sqrt(vx**2 + vy**2)
    if speed < 1e-6: ax_drag, ay_drag = 0, 0
    else:
        ax_drag = -k_d * vx * speed; ay_drag = -k_d * vy * speed
    ax_net = ax_cmd + ax_drag; ay_net = ay_cmd + ay_drag
    px_next = px + vx * dt + 0.5 * ax_net * dt**2
    py_next = py + vy * dt + 0.5 * ay_net * dt**2
    vx_next = vx + ax_net * dt; vy_next = vy + ay_net * dt
    return np.array([px_next, py_next, vx_next, vy_next])
def measurement_function(x): return np.array([x[0], x[1]])

# --- Pygame 및 2D 시뮬레이션 파라미터 ---
pygame.init()
WIDTH, HEIGHT = 1000, 800
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("2D UKF 시뮬레이션 (원형 경로 + 속도 보상)")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 28)

# 두 번째 코드와 동일한 파라미터 사용
dt = 1.0; accel_noise_std = 0.02; measurement_noise_std = 5.0
num_steps = 600; K_DRAG = 0.001

# 원형 궤도 및 속도 제어 파라미터
ORBIT_CENTER_X = WIDTH / 2.0; ORBIT_CENTER_Y = HEIGHT / 2.0
ORBIT_RADIUS = min(WIDTH, HEIGHT) / 3.0
TARGET_ANGULAR_VELOCITY = 0.04
V_TARGET = TARGET_ANGULAR_VELOCITY * ORBIT_RADIUS
ACCEL_C_MAG_CMD = TARGET_ANGULAR_VELOCITY**2 * ORBIT_RADIUS
KP_SPEED = 0.1

# 노이즈 행렬
Q_diag = np.array([(0.5*dt**2)**2*accel_noise_std**2, (0.5*dt**2)**2*accel_noise_std**2, (dt)**2*accel_noise_std**2, (dt)**2*accel_noise_std**2])
Q = np.diag(Q_diag); R = np.eye(2) * measurement_noise_std**2

# 초기 상태
initial_px = ORBIT_CENTER_X + ORBIT_RADIUS; initial_py = ORBIT_CENTER_Y
initial_vx = 0.0; initial_vy = V_TARGET
initial_true_state = np.array([initial_px, initial_py, initial_vx, initial_vy])
x_est_initial = np.array([initial_px + 15, initial_py - 15, initial_vx, initial_vy - 0.5])
P_initial = np.eye(4) * 50.0

# ==============================================================
# 단계 1 & 2: 시뮬레이션 및 UKF 적용 (통합 루프)
# ==============================================================
print("시뮬레이션 및 UKF 적용 중...")
true_states = []; estimated_states = []; measurements_for_viz = []
current_true_state = initial_true_state.copy()

ukf = UnscentedKalmanFilter(
    x0=x_est_initial, P=P_initial, Q=Q, R=R, dt=dt,
    fx=lambda x, u, d: state_transition_function(x, u, d, K_DRAG),
    hx=measurement_function
)

for k in range(num_steps):
    # --- '제어 입력 u_k' 계산 ---
    px_curr, py_curr, vx_curr, vy_curr = current_true_state
    dx_to_center = ORBIT_CENTER_X - px_curr; dy_to_center = ORBIT_CENTER_Y - py_curr
    dist_to_center = np.sqrt(dx_to_center**2 + dy_to_center**2)
    ax_centripetal = 0.0; ay_centripetal = 0.0
    if dist_to_center > 1e-6:
        norm_x = dx_to_center / dist_to_center; norm_y = dy_to_center / dist_to_center
        ax_centripetal = ACCEL_C_MAG_CMD * norm_x; ay_centripetal = ACCEL_C_MAG_CMD * norm_y
    
    current_speed = np.sqrt(vx_curr**2 + vy_curr**2); speed_error = V_TARGET - current_speed
    ax_tangential = 0.0; ay_tangential = 0.0
    if current_speed > 1e-6:
        norm_vx = vx_curr / current_speed; norm_vy = vy_curr / current_speed
        accel_tangential_scalar = KP_SPEED * speed_error
        ax_tangential = accel_tangential_scalar * norm_vx; ay_tangential = accel_tangential_scalar * norm_vy
        
    u_k = np.array([ax_centripetal + ax_tangential, ay_centripetal + ay_tangential])
    
    # --- 실제 경로 업데이트 ---
    process_noise = np.random.multivariate_normal(np.zeros(4), Q)
    current_true_state = state_transition_function(current_true_state, u_k, dt, K_DRAG) + process_noise
    true_states.append(current_true_state.copy())

    # --- UKF 업데이트 ---
    # 필터에도 동일한 '똑똑한 제어 입력' u_k를 알려줌
    ukf.predict(u=u_k)
    measurement_noise = np.random.multivariate_normal(np.zeros(2), R)
    measurement_k = measurement_function(current_true_state) + measurement_noise
    measurements_for_viz.append(measurement_k)
    ukf.update(measurement_k)
    estimated_states.append(ukf.x.copy())

# ==============================================================
# 단계 3: 시각화 (측정값 포함)
# ==============================================================
print("단계 3: 시각화 시작...")
running = True; frame_index = 0
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT: running = False
    screen.fill((255, 255, 255))
    pygame.draw.circle(screen, (220, 220, 220), (int(ORBIT_CENTER_X), int(ORBIT_CENTER_Y)), int(ORBIT_RADIUS), 1)

    if frame_index < num_steps:
        true_pos = (int(true_states[frame_index][0]), int(true_states[frame_index][1]))
        est_pos = (int(estimated_states[frame_index][0]), int(estimated_states[frame_index][1]))
        meas_pos = (int(measurements_for_viz[frame_index][0]), int(measurements_for_viz[frame_index][1]))

        if frame_index > 0:
            prev_true_pos = (int(true_states[frame_index-1][0]), int(true_states[frame_index-1][1]))
            pygame.draw.line(screen, (0, 0, 200), prev_true_pos, true_pos, 2)
            prev_est_pos = (int(estimated_states[frame_index-1][0]), int(estimated_states[frame_index-1][1]))
            pygame.draw.line(screen, (200, 0, 0), prev_est_pos, est_pos, 2)
            
        pygame.draw.circle(screen, (180, 180, 180), meas_pos, 6)
        pygame.draw.circle(screen, (0, 0, 255), true_pos, 8)
        pygame.draw.circle(screen, (255, 0, 0), est_pos, 8, 3)

        text_surface = font.render(f"스텝: {frame_index+1}/{num_steps}", True, (0,0,0))
        screen.blit(text_surface, (10, 10))
        frame_index += 1
    else:
        font_large = pygame.font.Font(None, 40)
        end_text = font_large.render("UKF (원형 CMD, 속도 보상) 시뮬레이션 완료", True, (0, 100, 0))
        screen.blit(end_text, end_text.get_rect(center=(WIDTH/2, HEIGHT/2)))

    pygame.display.flip()
    clock.tick(60)
pygame.quit()

시뮬레이션 및 UKF 적용 중...
단계 3: 시각화 시작...
